In [1]:
# Jun's code for ANN training with pinball loss
# import functions
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
# auxiliary_torch contains all the user-defined functions
from auxiliary_torch import pinball_loss, train_model_batch_saveloss
import matplotlib.pyplot as plt
import matplotlib as mpl

/Users/jhao2/opt/anaconda3/envs/nwbd/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#%% data preprocessing
# load data
cols = ['kw','ws','pos','windspeed_200m','temperature_200m','pressure_200m','pressure_100m']
# train_file = ['path/to/file/All_combo1.xlsx']
train_file = ['Data/Path_to_train_file.xlsx']
train_data = []
test_data = []

In [3]:
# load preprocessed data
for i in range(len(train_file)):
    train_data.append(pd.read_excel(train_file[i], sheet_name='kw_ws_new'))

In [4]:
ratio=8
seed=46
data_set = pd.concat(train_data, axis=0, ignore_index=True)
data_set_1 = data_set[cols]
# the one line code below is optional in case a model of specific wind region is needed
data_set_1 = data_set_1.drop(data_set_1[data_set_1.ws<12].index)
data_set_1 = data_set_1.values
X_data = data_set_1[:,1:]
y_data = data_set_1[:,0]
test_size = 0.30
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=test_size, random_state=seed)

In [5]:
# normalize data
min_max_scaler = preprocessing.MinMaxScaler()
X_train = min_max_scaler.fit_transform(X_train)
X_test = min_max_scaler.transform(X_test)

In [6]:
# quantiles
quantiles = np.arange(1, 100)
targets = np.percentile(y_train, quantiles)

In [7]:
# prepare the data for torch
X_train = torch.from_numpy(X_train)
print("X_train tensor size", X_train.size())
X_test = torch.from_numpy(X_test)
y_train = torch.from_numpy(y_train)
print("y_train tensor size", y_train.size())
y_test = torch.from_numpy(y_test)

Train_set = TensorDataset(X_train, y_train)
Test_set = TensorDataset(X_test,y_test)

batch_size = 64
data_loader_train = DataLoader(Train_set, batch_size, shuffle=True)
data_loader_test = DataLoader(Test_set, batch_size, shuffle=True)

X_train tensor size torch.Size([2284, 6])
y_train tensor size torch.Size([2284])


In [8]:
#%% Pytorch model
# Define the neural network architecture
class QuantileRegression(nn.Module):
    def __init__(self, n_features, n_quantiles):
        super(QuantileRegression, self).__init__()
        torch.set_default_dtype(torch.float64)
        self.n_quantiles = n_quantiles
        self.layer1 = nn.Linear(n_features, 64)
        self.layer2 = nn.Linear(64, 64)
        self.layer3 = nn.Linear(64, n_quantiles)

    def forward(self, x):
        x = nn.functional.relu(self.layer1(x))
        x = nn.functional.relu(self.layer2(x))
        x = self.layer3(x)
        return x

# Define the quantiles to predict
quantiles = torch.linspace(0.01, 0.99, 99)

In [9]:
# Instantiate the model
model = QuantileRegression(n_features=X_train.shape[1], n_quantiles=quantiles.shape[0])
optimizer = optim.Adam(model.parameters(), lr=0.0001)
num_epochs=200
# Train the model
loss_train, loss_validate = train_model_batch_saveloss(model, optimizer, data_loader_train, data_loader_test, quantiles, num_epochs=num_epochs)

Epoch [10/200], Loss: 738.2068
Epoch [20/200], Loss: 729.7114
Epoch [30/200], Loss: 706.6194
Epoch [40/200], Loss: 664.8803
Epoch [50/200], Loss: 601.7505
Epoch [60/200], Loss: 514.9913
Epoch [70/200], Loss: 403.1012
Epoch [80/200], Loss: 264.5970
Epoch [90/200], Loss: 111.6625
Epoch [100/200], Loss: 58.2535
Epoch [110/200], Loss: 52.5179
Epoch [120/200], Loss: 50.9685
Epoch [130/200], Loss: 50.0449
Epoch [140/200], Loss: 49.5105
Epoch [150/200], Loss: 49.1091
Epoch [160/200], Loss: 48.6728
Epoch [170/200], Loss: 48.1713
Epoch [180/200], Loss: 47.8011
Epoch [190/200], Loss: 47.2212
Epoch [200/200], Loss: 46.6797


In [10]:
# plot loss curve
mpl.rcParams['lines.linewidth'] = 4
plt.rcParams.update({'font.size': 16})
mpl.rcParams['figure.dpi'] = 1200

plt.plot(np.linspace(1, num_epochs, num_epochs).astype(int), loss_train, color='lightskyblue', label='Training loss')
plt.plot(np.linspace(1, num_epochs, num_epochs).astype(int), loss_validate, color='red', linestyle=(0, (3, 1, 1, 1, 1, 1)), label='Validation loss')
plt.title('Loss curve test size '+str(test_size*100)+' percents')
plt.ylabel('Loss')
plt.xlabel('Number of epochs')
plt.legend()
plt.tight_layout()
fig = plt.gcf()
fig.set_size_inches(12.5, 7.5)
plt.show()
#fig.savefig('result/PC/ANN/trainvallosscurve_ws12_'+str(int(test_size*100))+'percents.png',dpi=1200)b

In [11]:
# Evaluate the model
with torch.no_grad():
    preds = model(X_test)
    loss = pinball_loss(preds, y_test, quantiles)
    print(f'Validation Loss: {loss.item():.4f}')

Validation Loss: 47.3267


In [12]:
#%% optional save with ws
concatenated_output = torch.cat((X_test[:,0].view(-1,1), y_test.view(-1,1), preds), dim=1)
df = pd.DataFrame(data = concatenated_output.numpy(), columns=['ws','Actual']+['Q'+str(k) for k in range(1,100)])
X_test1 = min_max_scaler.inverse_transform(X_test.numpy())
df['ws']=X_test1[:,0]
# save the DataFrame as an Excel file
#df.to_excel("path/to/ur/location/output_test_ws12"+str(int(test_size*100))+"percents.xlsx", index=False)
df.to_excel("result/PC/ANN/output_test_ws12"+str(int(test_size*100))+"percents.xlsx", index=False)